In [3]:
!pip install pyspark==3.0.1 py4j==0.10.9


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, asc, desc, regexp_replace
from pyspark.sql import SparkSession, DataFrame


spark = SparkSession.builder.appName("SparkTest").getOrCreate()

# В датафреймах заданы продукты, категории и связь между ними.
products_data = [(1, "product1"),
                 (2, "product2"),
                 (3, "product3"),
                 (4, "product4")]
categories_data = [(1, "category1"),
                   (2, "category2"),
                   (3, "category3"),
                   (4, "category4")]
products_categories_data = [(1, 1),
                            (1, 2),
                            (2, 1),
                            (4, 3),
                            (4, 2)]

schema_p = StructType([StructField("id", IntegerType(), False), StructField("product_name", StringType(), False)])
schema_c = StructType([StructField("id", IntegerType(), False), StructField("category_name", StringType(), False)])
schema_pc = StructType([StructField("id_product", IntegerType(), True), StructField("id_category", IntegerType(), True)])

df_p = spark.createDataFrame(data=products_data,schema=schema_p)
df_c = spark.createDataFrame(data=categories_data,schema=schema_c)
df_pc = spark.createDataFrame(data=products_categories_data,schema=schema_pc)

# Датафрейм с набором всех пар «Имя продукта – Имя категории»
df_rez = (df_p.join(df_pc, df_p.id == df_pc.id_product, how='left_outer')).join(df_c, df_c.id == df_pc.id_category, how ='left_outer').orderBy('product_name','category_name')
df_rez = df_rez.select(col("product_name").alias("Product"), col("category_name").alias("Category"))
df_rez.show()

Exception: Java gateway process exited before sending its port number